# Data Access

Taken from a notebook created by Tyler Sutterly for the 2022 ICESat-2 Hackweek, put on by the e-science institute at UW.  
Use conda env sliderule  

Install new version of sliderule:  
"But if you do a **git pull** in your sliderule-python repo and then to a **python setup.py install**, it should resolve the issue."  
https://icesat-2sliderule.slack.com/archives/CTW2ESWFP/p1638199956016000

In [1]:
import os
import geopandas as gpd
import ipyleaflet
import ipywidgets as widgets
import logging
#import panel as pn
#pn.extension()
import matplotlib.lines
import matplotlib.colors as colors
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
import numpy as np
from shapely.geometry import Polygon
import warnings
# grimp and nisar functions
#import grimpfunc as grimp
#import nisardev as nisar
# sliderule functions
import sliderule.icesat2
import sliderule.io
import sliderule.ipysliderule
import pandas as pd
from datetime import datetime
# register progress bar and set workers
#ProgressBar().register()
#dask.config.set(num_workers=2)
# turn off warnings for demo
warnings.filterwarnings('ignore')

In [2]:
%matplotlib inline

In [3]:
# set the url for the sliderule service
# set the logging level
sliderule.icesat2.init("icesat2sliderule.org", loglevel=logging.WARNING)

In [4]:
SRwidgets?

Object `SRwidgets` not found.


In [5]:
# display widgets for setting SlideRule parameters
SRwidgets = sliderule.ipysliderule.widgets()
# show widgets
widgets.VBox([
    SRwidgets.asset,
    SRwidgets.release,
    SRwidgets.surface_type,
    SRwidgets.length,
    SRwidgets.step,
    SRwidgets.confidence,
    SRwidgets.land_class,
    SRwidgets.iteration,
    SRwidgets.spread,
    SRwidgets.count,
    SRwidgets.window,
    SRwidgets.sigma
])

In [7]:
widgets.VBox([SRwidgets.projection, SRwidgets.layers])

In [9]:
pwd

'/home/jovyan/isce_sat2/contributors/naheem'

In [12]:
polygon_gdf = gpd.read_file('grandmesa.geojson', crs='epsg:4326')
polygon_gdf.head()

,geometry
0,"POLYGON ((-108.31168 39.13758, -108.34116 39.0..."


In [11]:
polygon_gdf = polygon_gdf[1:]

In [ ]:
gdf_exploded = polygon_gdf.explode()

In [13]:
polygon_gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
gdf_exploded.plot()

In [14]:
gdf_exploded.type

NameError: name 'gdf_exploded' is not defined

In [15]:
# create ipyleaflet map in specified projection
m = sliderule.ipysliderule.leaflet(SRwidgets.projection.value)
m.add_layer(layers=SRwidgets.layers.value)

# Comment this section out if you want to draw your own polygon!
# ---
#box = shapely.geometry.Polygon([(-108.3,38.9), (-108.0,38.9), (-108.0,39.1), (-108.3, 39.1)])
#geobox = gpd.GeoDataFrame(geometry=[box], crs='EPSG:4326')
default_polygon = sliderule.io.from_geodataframe(polygon_gdf)
geodata = ipyleaflet.GeoData(geo_dataframe=polygon_gdf)
m.map.add_layer(geodata)
# ---

m.map

Map(center=[39, -108], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [17]:
#%%time
# sliderule asset and data release
asset = SRwidgets.asset.value
release = SRwidgets.release.value

# build sliderule parameters using latest values from widget
params = {
    # surface type: 0-land, 1-ocean, 2-sea ice, 3-land ice, 4-inland water
    "srt": SRwidgets.surface_type.index,
    # length of ATL06-SR segment in meters
    "len": SRwidgets.length.value,
    # step distance for successive ATL06-SR segments in meters
    "res": SRwidgets.step.value,
    # confidence level for PE selection
    "cnf": SRwidgets.confidence.value,
    # ATL08 land surface classifications
    "atl08_class": list(SRwidgets.land_class.value),
    # maximum iterations, not including initial least-squares-fit selection
    "maxi": SRwidgets.iteration.value,
    # minimum along track spread
    "ats": SRwidgets.spread.value,
    # minimum PE count
    "cnt": SRwidgets.count.value,
    # minimum height of PE window in meters
    "H_min_win": SRwidgets.window.value,
    # maximum robust dispersion in meters
    "sigma_r_max": SRwidgets.sigma.value
}

region = sliderule.icesat2.toregion(polygon_gdf)
params["poly"] = region["poly"]   # will always be the convex hull of the polygon
params["raster"] = region["raster"]   # raster will include all vertices, including holes. This is more processing, but is only an issue with very large requests.
gdf = sliderule.icesat2.atl06p(params, asset, version=release)

In [18]:
print(f'Returned {gdf.shape[0]} records')
gdf.head()

Returned 94078 records


,gt,rgt,h_mean,n_fit_photons,rms_misfit,delta_time,spot,distance,dh_fit_dy,dh_fit_dx,w_surface_window_final,pflags,h_sigma,segment_id,cycle,geometry
time,,,,,,,,,,,,,,,,
2018-10-17 22:31:17.332951552,60,295,1821.473855,59,0.186843,2.505068e+07,1,4.326519e+06,0.0,0.020385,3.0,0,0.024414,215739,1,POINT (-108.28616 38.88850)
2018-10-17 22:31:17.338676880,60,295,1822.970517,12,0.225525,2.505068e+07,1,4.326559e+06,0.0,0.052908,3.0,0,0.084989,215741,1,POINT (-108.28620 38.88887)
2018-10-17 22:31:17.350047904,60,295,1826.151552,32,0.287631,2.505068e+07,1,4.326639e+06,0.0,0.019818,3.0,0,0.098879,215745,1,POINT (-108.28629 38.88959)
2018-10-17 22:31:17.355689616,60,295,1827.168388,82,0.235445,2.505068e+07,1,4.326680e+06,0.0,0.034429,3.0,0,0.026206,215747,1,POINT (-108.28633 38.88995)
2018-10-17 22:31:17.361279052,60,295,1827.834753,63,0.243145,2.505068e+07,1,4.326720e+06,0.0,-0.011353,3.0,0,0.032105,215749,1,POINT (-108.28638 38.89031)


In [19]:
gdf.tail()

,gt,rgt,h_mean,n_fit_photons,rms_misfit,delta_time,spot,distance,dh_fit_dy,dh_fit_dx,w_surface_window_final,pflags,h_sigma,segment_id,cycle,geometry
time,,,,,,,,,,,,,,,,
2022-03-09 11:21:16.902325984,30,1179,3141.000499,11,0.000000,1.320601e+08,3,4.345455e+06,0.0,0.844762,30.000000,2,0.000000,216687,14,POINT (-107.85255 39.05339)
2022-03-09 11:21:17.108213888,10,1179,3088.928212,14,0.597828,1.320601e+08,1,4.346798e+06,0.0,0.041497,3.644908,0,0.159807,216754,14,POINT (-107.89063 39.06276)
2022-03-09 11:21:17.130550352,10,1179,3101.792783,11,1.642566,1.320601e+08,1,4.346958e+06,0.0,-0.108945,4.126904,4,0.516383,216762,14,POINT (-107.89082 39.06419)
2022-03-09 11:21:17.136199872,10,1179,3102.247719,20,2.243868,1.320601e+08,1,4.346998e+06,0.0,0.157041,8.542609,4,0.505032,216764,14,POINT (-107.89086 39.06455)
2022-03-09 11:21:17.141899664,10,1179,3110.133520,14,2.694566,1.320601e+08,1,4.347038e+06,0.0,0.270824,9.757998,4,0.724263,216766,14,POINT (-107.89090 39.06491)


In [20]:
gdf.index.is_unique

True

In [ ]:
#gdf.time = pd.DatetimeIndex(gdf.time)
#gdf = gdf.set_index(['time'])

# gdf.sort_index(inplace=True)
# gdf['acqdate'] = gdf.index.date
# gdf['acqdate_string'] = gdf['acqdate'].apply(lambda x: datetime.strftime(x, '%Y-%m-%d'))

In [ ]:
#gdf = gdf.drop(columns=['acqdate','acqdate_string'])

In [21]:
gdf.head(2)

,gt,rgt,h_mean,n_fit_photons,rms_misfit,delta_time,spot,distance,dh_fit_dy,dh_fit_dx,w_surface_window_final,pflags,h_sigma,segment_id,cycle,geometry
time,,,,,,,,,,,,,,,,
2018-10-17 22:31:17.332951552,60,295,1821.473855,59,0.186843,2.505068e+07,1,4.326519e+06,0.0,0.020385,3.0,0,0.024414,215739,1,POINT (-108.28616 38.88850)
2018-10-17 22:31:17.338676880,60,295,1822.970517,12,0.225525,2.505068e+07,1,4.326559e+06,0.0,0.052908,3.0,0,0.084989,215741,1,POINT (-108.28620 38.88887)


In [22]:
gdf.acqdate.unique()

AttributeError: 'GeoDataFrame' object has no attribute 'acqdate'

In [23]:
gdf['elev_plus'] = gdf['h_mean'] + 1

In [24]:
widgets.VBox([
    SRwidgets.variable,
    SRwidgets.cmap,
    SRwidgets.reverse,
])

In [25]:
widgets.DatePicker(
    description='Pick a Date',
    disabled=False
)


DatePicker(value=None, description='Pick a Date')

In [26]:
start_date = datetime(2018, 11, 10)
end_date = datetime(2022, 3, 1)

dates = pd.date_range(start_date, end_date, freq='D')

options = [(date.strftime(' %d %b %Y '), date) for date in dates]
index = (0, len(options)-1)

selection_range_slider = widgets.SelectionRangeSlider(
    options=options,
    index=index,
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

selection_range_slider


SelectionRangeSlider(description='Dates', index=(0, 1207), layout=Layout(width='500px'), options=((' 10 Nov 20…

In [28]:
SRwidgets

In [29]:
m.GeoData(gdf, column_name=SRwidgets.variable.value, cmap='viridis',
    max_plot_points=10000, tooltip=True, colorbar=True)

In [30]:
SRwidgets.

SyntaxError: invalid syntax (3762063117.py, line 1)

In [31]:
date = dict(zip(gdf[''].tolist(), africa[‘pop_est’].tolist()))
africa[[“name”, “geometry”]].to_file(“data/africa.json”, driver=”GeoJSON”, encoding=’utf-8')
geojson_data = json.load(open(“data/africa.json”,’r’))for feature in geojson_data[‘features’]:
     properties = feature[‘properties’]
     feature.update(id=properties[‘name’])
     #print(feature[‘id’])

SyntaxError: invalid character '‘' (U+2018) (853659073.py, line 1)

#### Create plots for a single track

In [32]:
# selection for reference ground track
RGTs = [str(int(x)) for x in gdf.rgt.unique()]
SRwidgets.rgt = widgets.Dropdown(
    options=RGTs,
    value=RGTs[1], # You can change this with the pull-down widget below!
    description="RGT:",
    description_tooltip="RGT: Reference Ground Track to plot",
    disabled=False
)

# selection for ground track
ground_track_options = ["gt1l","gt1r","gt2l","gt2r","gt3l","gt3r"]
SRwidgets.ground_track = widgets.Dropdown(
    options=ground_track_options,
    value='gt1l',
    description="Track:",
    description_tooltip="Track: Ground Track to plot",
    disabled=False
)      
widgets.VBox([
    SRwidgets.rgt,
    SRwidgets.ground_track,
])

In [33]:
gdf.index

DatetimeIndex(['2018-10-17 22:31:17.332951552',
               '2018-10-17 22:31:17.338676880',
               '2018-10-17 22:31:17.350047904',
               '2018-10-17 22:31:17.355689616',
               '2018-10-17 22:31:17.361279052',
               '2018-10-17 22:31:17.366855968',
               '2018-10-17 22:31:18.140942396',
               '2018-10-17 22:31:18.146572856',
               '2018-10-17 22:31:18.152203580',
               '2018-10-17 22:31:18.240796700',
               ...
               '2022-03-07 23:39:21.897285824',
               '2022-03-07 23:39:22.050853024',
               '2022-03-07 23:39:22.388791296',
               '2022-03-07 23:39:22.620028464',
               '2022-03-09 11:21:16.897794800',
               '2022-03-09 11:21:16.902325984',
               '2022-03-09 11:21:17.108213888',
               '2022-03-09 11:21:17.130550352',
               '2022-03-09 11:21:17.136199872',
               '2022-03-09 11:21:17.141899664'],
              dtype=

In [34]:
gdf.index = gdf.index.astype(str)

In [35]:
type(gdf.index)

pandas.core.indexes.base.Index

In [36]:
!pwd

/home/jovyan/isce_sat2/contributors/naheem


In [37]:
fn = '../../data/tuolumne_all_sr-atl06-07072022.geojson'

In [ ]:
gdf.to_file(fn, driver='GeoJSON')

In [ ]:
gdf2 = gpd.read_file(fn)

In [ ]:
gdf2.head()

In [ ]:
type(gdf2.index)

In [ ]:
gdf2['time'] = pd.to_datetime(gdf2['time'])

In [ ]:
gdf2['time'].is_unique

In [ ]:
gdf2['time'].is_unique

In [ ]:
gdf2.loc[gdf2['time'].duplicated(), :]

In [ ]:
gdf2['time'].duplicated().sum()

In [ ]:
gdf2.head()

In [ ]:
gdf2.index.is_unique

In [ ]:
gdf2.time = pd.DatetimeIndex(gdf2.time)

In [ ]:
gdf2 = gdf2.set_index(['time'])
gdf2.sort_index(inplace=True)

In [ ]:
gdf2.index.is_unique

In [ ]:
type(gdf2.time[0])

In [ ]:
fig, ax = plt.subplots()
gdf.plot(ax=ax)
polygon_gdf.plot(ax=ax)